# Part 1: Data importing and preprocessing

In [ ]:
# Imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read in file
house_df = pd.read_csv("Data/house_sales.csv")
house_df.head()

In [ ]:
# Determine null values
house_df.isnull().sum()

### Missing values for sqft_living

In [ ]:
# Fill missing values for sqft_living
house_df['sqft_living'] = house_df['sqft_living'].fillna(house_df['sqft_above'] + house_df['sqft_basement'])

# Check for missing values
print(f"sqft_living missing values: {house_df['sqft_living'].isnull().sum()}")

### Missing values for bedrooms and bathrooms

In [ ]:
# Use sqft_living bins to fill missing values for bedroom and bathroom
bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, float('inf')]
labels = ['0-1000', '1000-2000', '2000-3000', '3000-4000', '4000-5000', '5000-6000', '6000+']
house_df['sqft_bin'] = pd.cut(house_df['sqft_living'], bins=bins, labels=labels)

In [ ]:
# Drop rows with missing values in the relevant columns just for the calculation
grouped_avgs = house_df.dropna(subset=['bedrooms', 'bathrooms']).groupby('sqft_bin')[['bedrooms', 'bathrooms']].mean().round()
print(grouped_avgs)

In [ ]:
# Create dictionaries from the grouped_avgs to map bins to fill missing values with
bed_avg = grouped_avgs['bedrooms'].to_dict()
bath_avg = grouped_avgs['bathrooms'].to_dict()

# Fill missing values for bedrooms
house_df.loc[house_df['bedrooms'].isna(), 'bedrooms'] = (
    house_df.loc[house_df['bedrooms'].isna(), 'sqft_bin'].map(bed_avg))

# Fill missing values for bathrooms
house_df.loc[house_df['bathrooms'].isna(), 'bathrooms'] = (
    house_df.loc[house_df['bathrooms'].isna(), 'sqft_bin'].map(bath_avg))

In [ ]:
# Check for missing values
print(f"bedrooms missing values: {house_df['bedrooms'].isnull().sum()}")
print(f"bathrooms missing values: {house_df['bathrooms'].isnull().sum()}")

### Checking bedroom outliers

In [ ]:
# Checking bedrooms for outliers
print(f"bedroom values: {sorted(house_df['bedrooms'].unique())}")

In [ ]:
# Bedrooms boxplot
sns.boxplot(house_df['bedrooms'])

In [ ]:
# Count of how many homes have an outlier value of bedrooms
print(f"Over 10 bedrooms: {(house_df['bedrooms'] > 10).sum()}")
print(f"10 bedrooms: {(house_df['bedrooms'] == 10).sum()}")
print(f"9 bedrooms: {(house_df['bedrooms'] == 9).sum()}")
print(f"8 bedrooms: {(house_df['bedrooms'] == 8).sum()}")
print(f"7 bedrooms: {(house_df['bedrooms'] == 7).sum()}")
print(f"6 bedrooms: {(house_df['bedrooms'] == 6).sum()}")
print(f"1 bedrooms: {(house_df['bedrooms'] == 1).sum()}")
print(f"0 bedrooms: {(house_df['bedrooms'] == 0).sum()}")

In [ ]:
# Drop outlier values of bedrooms (0, 7, 8, 9, 10, 33)
bedroom_clean = house_df.drop(house_df[house_df['bedrooms'].isin([0, 7, 8, 9, 10, 33])].index)
sorted(bedroom_clean['bedrooms'].unique())

### Checking bathroom outliers

In [ ]:
# Checking bathrooms for outliers
print(f"bathroom values: {sorted(bedroom_clean['bathrooms'].unique())}")

In [ ]:
# Adjusting bathroom values to be whole or half (no 0.25 or 0.75 values)
def clean_bathroom_value(x):
    # 0.25 values will be rounded down to nearest whole value
    decimal = x % 1
    if decimal == 0.25: 
        return np.floor(x) # floor rounds down
    # 0.75 values will be rounded up to nearest whole value
    elif decimal == 0.75:
        return np.ceil(x) # ceil rounds up
    else:
        return x

# Apply function
bedroom_clean['bathrooms'] = bedroom_clean['bathrooms'].apply(clean_bathroom_value)

# Check if function worked
print(f"bathroom values: {sorted(bedroom_clean['bathrooms'].unique())}")

In [ ]:
# Bathrooms boxplot
sns.boxplot(bedroom_clean['bathrooms'])

In [ ]:
# Count of how many homes have an outlier value of bathrooms
print(f"More than 6 bathrooms: {(bedroom_clean['bathrooms'] > 6).sum()}")
print(f"Less than 1 bathroom: {(bedroom_clean['bathrooms'] < 1).sum()}")

In [ ]:
# Drop outlier values for bathrooms
bathroom_clean = bedroom_clean[(bedroom_clean['bathrooms'] >= 1) & (bedroom_clean['bathrooms'] <= 6)]
sorted(bathroom_clean['bathrooms'].unique())

### Missing values for sqft_lot

In [ ]:
# Replace missing 'sqft_lot' values with the averaged 'sqft_lot15' values
bathroom_clean['sqft_lot'] = bathroom_clean['sqft_lot'].fillna(bathroom_clean['sqft_lot15'])

print(f"sqft_lot missing values: {bathroom_clean['sqft_lot'].isnull().sum()}")

In [ ]:
# Ensure no other values in the df are null
bathroom_clean.isnull().sum()

### New dataframe with adjustments made

In [ ]:
# Create new clean house dataframe with changes made
house_clean = bathroom_clean[['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                              'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement',
                              'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long']]

In [ ]:
house_clean

In [ ]:
house_clean.info()

In [ ]:
# Convert date into datetime dtype
house_clean['date'] = pd.to_datetime(house_clean['date'], format='%Y%m%dT%H%M%S')

# Remove the hours, minutes, and second -- keep only year, month, day
house_clean['date'] = house_clean['date'].dt.date

In [ ]:
# Convert bedrooms, sqft_living, and sqft_lot into int
house_clean['bedrooms'] = house_clean['bedrooms'].astype('int64')
house_clean['sqft_living'] = house_clean['sqft_living'].astype('int64')
house_clean['sqft_lot'] = house_clean['sqft_lot'].astype('int64')

In [ ]:
# Run correlations
house_w_basements = house_clean[(house_clean['sqft_basement'] > 0)] # Remove houses without basement
house_corr = house_w_basements[['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement']]
house_corr.corr()

In [ ]:
# Import new dataframe into csv
house_clean.to_csv('Data/house_updated.csv', index=False)

# Part 2: Data analysis and visualizations

In [ ]:
# Read in file
house_updated_df = pd.read_csv("Data/house_updated.csv")
house_updated_df.head()

In [ ]:
# Determine dtypes
house_updated_df.dtypes

Numerical Variables: id, price, bedrooms, bathrooms, sqft_living, sqft_lot, floors, sqft_above, sqft_basement, yr_built, yr_renovated, zipcode, lat, long
Continous: price, sqft_living, floors, sqft_above, sqft_basement, lat, long
Discrete: id, bedrooms, yr_built, yr_renovated, zipcode
Categorical variables: waterfront , view, condition, zipcode
Ordinal Variables: grade
Date Variable: Date

In [ ]:
# Create dateframe to show the dtype
house_data = {
    'Column': ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'],
    'Type': ['Numerical - Discrete', 'Date', 'Numerical - Continuous', 'Numerical - Discrete', 'Numerical - Continuous', 'Numerical - Continuous', 'Numerical - Continuous', 'Numerical - Continuous', 'Categorical - Binary', 'Categorical - Binary', 'Categorical - Nominal',
             'Ordinal - Ranked', 'Numerical - Continuous', 'Numerical - Continuous', 'Numerical - Discrete', 'Numerical -Discrete', 'Numerical/Categorical', 'Numerical - Continuous', 'Numerical - Continuous', 'Numerical - Continuous', 'Numerical - Continuous'],
}
house_data_variables_type_df = pd.DataFrame(house_data)
house_data_variables_type_df

In [ ]:
# Describe dataframe
house_MC_df = house_updated_df.describe()
house_MC_df

In [ ]:
# Measures of centrality: Price

In [ ]:
# Measures of centrality: Bedrooms

In [ ]:
# Measures of centrality: Bathrooms

In [ ]:
# Measures of centrality: SQFT_Living

In [ ]:
# Measures of centrality: SQFT_Lot

In [ ]:
# Measures of centrality: Floors

In [ ]:
# Measures of centrality: Condition

In [ ]:
# Measures of centrality: Grade

In [ ]:
# Measures of centrality: SQFT_Above

In [ ]:
# Measures of centrality: SQFT_Basement

In [ ]:
# Measures of centrality: Yr_Built

In [ ]:
# Measures of centrality: Yr_Renovated

In [ ]:
# Combined measures

In [ ]:
# Summary of measures

In [ ]:
# Histogram of price
sns.histplot(house_updated_df['price'], kde = True, bins=60)
plt.title('House Price Distributions')
plt.xlabel('Price (In millions)')
plt.ylabel('Count')
plt.show()

In [ ]:
# Boxplot of grade and price
plt.figure(figsize = (10, 8))
sns.boxplot(x = 'grade', y = 'price', data = house_updated_df)
plt.title('Grade vs Price Boxplot')
plt.xlabel('Grade of house denoted by a number (grade)')
plt.ylabel('Price')
plt.show()

In [ ]:
# Boxplot og condition and price
sns.boxplot(x = 'condition', y = 'price', data = house_updated_df)
plt.title('Condition vs Price Boxplot')
plt.xlabel('Condition of House (condition) ')
plt.ylabel('Price')
plt.show()

In [ ]:
 # Histogram of yr_built
sns.histplot(house_updated_df['yr_built'], kde = True, bins=50)
plt.title('Distribution of the Year House was built')
plt.xlabel('Year House was Built')
plt.ylabel('Count')
plt.show()

In [ ]:
# Scatterplot of sqft_living and price
sns.scatterplot(x='sqft_living', y='price', data=house_updated_df)
plt.xlabel('Living Area in square Feet (sqft_living)')
plt.ylabel('Price of Home (price)')
plt.title('Scatter Plot of Price vs. sqft_living')
plt.show()

In [ ]:
# Corr

# Part 3: Data analytics